## Set Library Path

In [1]:
.libPaths("/share/korflab/home/viki/anaconda3/jupyter_nb/lib/R/library")

## Update Bioconductor

In [ ]:
if (!require("BiocManager", quietly = TRUE))
    install.packages("BiocManager")
BiocManager::install(version = "3.17")

## Install Packages

In [ ]:
# ComplexHeatmap
if (!require("BiocManager", quietly = TRUE))
    install.packages("BiocManager")
BiocManager::install("ComplexHeatmap")

# dplyr
install.packages("dplyr")

# tidyr
install.packages("tidyr")

# reshape2
install.packages("reshape2")

## Load Packages

In [3]:
library(ComplexHeatmap)
library(dplyr)
library(tidyr)
library(reshape2)
library(stringr)

## Load Dataframe

In [4]:
df <- read.csv("tables_and_figures/expected_vs_observed_peaks_master.csv", header = TRUE)

In [5]:
# Format data

# Select data of interest
heatmap_data <- df %>%
  select(Endedness, Peak_Type, Aligner, Peak_Caller, Deduplicator, Test_Dataset, Control, Sensitivity, Precision, F1_Score)

# Combine variables for plotting
heatmap_data$x_axis_labels <- paste(heatmap_data$Peak_Caller, heatmap_data$Deduplicator, heatmap_data$Test_Dataset, sep = "_")
heatmap_data$y_axis_labels <- paste(heatmap_data$Endedness, heatmap_data$Peak_Type, heatmap_data$Aligner, sep = "_")

# Replace NA values with 0
heatmap_data <- heatmap_data %>%
  mutate(Sensitivity = replace_na(Sensitivity, 0),
         Precision = replace_na(Precision, 0),
         F1_Score = replace_na(F1_Score, 0))

## Data with Control

### F1 Scores

#### Create matrix

In [6]:
# Subset for data with control
control_df <- heatmap_data[heatmap_data$Control == "with_control", ]

# Melt the data from the heatmap_data dataframe
melted_control_df <- melt(control_df, id.vars = c("x_axis_labels", "y_axis_labels"), measure.vars = "F1_Score")

# Cast the data into the desired format
cast_control_df <- dcast(melted_control_df, y_axis_labels ~ x_axis_labels, value.var = "value")

# Generate matrices
rownames(cast_control_df) <- cast_control_df$y_axis_labels
cast_control_df$y_axis_labels <- NULL
control_matrix <- as.matrix(cast_control_df)

# Transpose matrix to switch rows and columns
control_matrix <- t(control_matrix)

#### Prepare metadata 

In [7]:
# Save matrix as data frame for manipulation
control_metadata <- as.data.frame(control_matrix)

# Peak Caller
new_row_names <- sub("^(.*?)_.*$", "\\1", rownames(control_matrix))
replacement_rules <- c("cisgenome" = "Cisgenome", "macs3" = "MACS3", "pepr" = "PePr", "genrich" = "Genrich")
new_row_names <- sapply(new_row_names, function(name) {
  if (name %in% names(replacement_rules)) {
    return(replacement_rules[name])
  } else {
    return(name)
  }
})
control_metadata$Peak_Caller <- new_row_names

# Deduplicator
new_row_names <- str_extract(rownames(control_matrix), "(?<=_)[^_]+")
replacement_rules <- c("no" = "None", "sambamba" = "Sambamba", "picard" = "Picard", "samtools" = "Samtools")
new_row_names <- sapply(new_row_names, function(name) {
  if (name %in% names(replacement_rules)) {
    return(replacement_rules[name])
  } else {
    return(name)
  }
})
control_metadata$Deduplicator <- new_row_names

# Test_Dataset
control_metadata$Test_Dataset <- as.integer(gsub(".*_(\\d+)$", "\\1", rownames(control_metadata)))
value_map <- c("4X", "8X", "16X", "32X", "64X", "128X")
control_metadata$Test_Dataset <- value_map[control_metadata$Test_Dataset]

#### Create heatmap

In [8]:
# Annotation objects

# Define the desired order for Peak Coverage levels
peak_coverage_order <- c("4X", "8X", "16X", "32X", "64X", "128X")

# Define the desired order for Endedness and Peak Type
desired_column_order <- c(
  "paired_broad_bowtie2", "paired_broad_bwa_mem", "paired_broad_STAR",
  "paired_narrow_bowtie2", "paired_narrow_bwa_mem", "paired_narrow_STAR",
  "single_broad_bowtie2", "single_broad_bwa_mem", "single_broad_STAR",
  "single_narrow_bowtie2", "single_narrow_bwa_mem", "single_narrow_STAR"
)

# Rearrange the columns in matrix
control_matrix_reordered <- control_matrix[, desired_column_order]

# Define custom grouping factor for column_split
n_blocks <- 4
n_columns_per_block <- 3
block_labels <- rep(1:n_blocks, each = n_columns_per_block)
block_split_factor <- factor(block_labels)

# Split rows by peak caller
row_split_factor <- factor(control_metadata$Peak_Caller)

# Row annotations
row_ha = rowAnnotation(`Peak Caller` = control_metadata$Peak_Caller,
                       `Deduplicator` = control_metadata$Deduplicator,
                       `Peak Coverage` = factor(control_metadata$Test_Dataset, levels = peak_coverage_order),
                       col = list(`Peak Caller` = c("Cisgenome" = "#c2ffc3","Genrich"= "#90ec7c","MACS3"= "#00b22a", "PePr" = "#006b05"),
                                  `Deduplicator` = c("Samtools" = "#620093", "None" = "#e6e6fa", "Picard" = "#aea3e0", "Sambamba" = "#835fbd"),
                                  `Peak Coverage` = c("4X" = "#00CCFF", "8X" = "#00A3FF", "16X" = "#007AFF", "32X" = "#0052FF", "64X" = "#0000FF", "128X" = "#0700C4"))
                        )

# Column annotations
col_ha = HeatmapAnnotation(`Endedness` = c(rep("Paired", 6), rep("Single", 6)),
                            `Peak Type` = c(rep("Broad", 3), rep("Narrow", 3), rep("Broad", 3), rep("Narrow", 3)),
                            `Aligner` = c("Bowtie2", "BWA-MEM", "STAR", "Bowtie2", "BWA-MEM", "STAR", "Bowtie2", "BWA-MEM", "STAR", "Bowtie2", "BWA-MEM", "STAR"),
                            col = list(`Endedness` = c("Paired" = "#f7d3db", "Single" = "#eb647e"),
                                       `Peak Type` = c("Narrow" = "#ff4d00", "Broad" = "#ff9a00"),
                                       `Aligner` = c("BWA-MEM" = "#8c0000", "Bowtie2" = "#ff0000", "STAR" = "#400000"))
                            )

In [9]:
# Create heatmap

pdf("tables_and_figures/f1_score_control_heatmap.pdf", width = 6, height = 10)

# Heatmap for control
control_heatmap_object <- Heatmap(
  # Data
  control_matrix, 
  
  # Column customizations
  cluster_columns = FALSE,
  show_column_names = FALSE,
  column_title = NULL,
  top_annotation = col_ha,
  column_split = block_split_factor,
  column_gap = unit(1, "mm"),
    
  # Row customizations
  cluster_rows = FALSE,
  show_row_names = FALSE,
  row_title = NULL,
  left_annotation = row_ha,
  row_split = row_split_factor,
  row_gap = unit(1, "mm"),

  # Legend customizations
  heatmap_legend_param = list(title = "F1 Score"),
  col = colorRampPalette(c("#B2182B", "#EEEEEE", "#2166AC"))(100),

  # Other customizations
  border = FALSE, 
  width = ncol(control_matrix)*unit(5, "mm"), 
  height = nrow(control_matrix)*unit(2, "mm")
)

ht = draw(control_heatmap_object)

# Print width and height to determine appropriate export size
w = ComplexHeatmap:::width(ht)
w = convertX(w, "inch", valueOnly = TRUE)
h = ComplexHeatmap:::height(ht)
h = convertY(h, "inch", valueOnly = TRUE)
c(w, h)

dev.off()

[1] 5.347920 9.540227

png 
  2

### Precision

#### Create matrix

In [10]:
# Subset for data with control
control_df <- heatmap_data[heatmap_data$Control == "with_control", ]

# Melt the data from the heatmap_data dataframe
melted_control_df <- melt(control_df, id.vars = c("x_axis_labels", "y_axis_labels"), measure.vars = "Precision")

# Cast the data into the desired format
cast_control_df <- dcast(melted_control_df, y_axis_labels ~ x_axis_labels, value.var = "value")

# Generate matrices
rownames(cast_control_df) <- cast_control_df$y_axis_labels
cast_control_df$y_axis_labels <- NULL
control_matrix <- as.matrix(cast_control_df)

# Transpose matrix to switch rows and columns
control_matrix <- t(control_matrix)

#### Prepare metadata 

In [11]:
# Save matrix as data frame for manipulation
control_metadata <- as.data.frame(control_matrix)

# Peak Caller
new_row_names <- sub("^(.*?)_.*$", "\\1", rownames(control_matrix))
replacement_rules <- c("cisgenome" = "Cisgenome", "macs3" = "MACS3", "pepr" = "PePr", "genrich" = "Genrich")
new_row_names <- sapply(new_row_names, function(name) {
  if (name %in% names(replacement_rules)) {
    return(replacement_rules[name])
  } else {
    return(name)
  }
})
control_metadata$Peak_Caller <- new_row_names

# Deduplicator
new_row_names <- str_extract(rownames(control_matrix), "(?<=_)[^_]+")
replacement_rules <- c("no" = "None", "sambamba" = "Sambamba", "picard" = "Picard", "samtools" = "Samtools")
new_row_names <- sapply(new_row_names, function(name) {
  if (name %in% names(replacement_rules)) {
    return(replacement_rules[name])
  } else {
    return(name)
  }
})
control_metadata$Deduplicator <- new_row_names

# Test_Dataset
control_metadata$Test_Dataset <- as.integer(gsub(".*_(\\d+)$", "\\1", rownames(control_metadata)))
value_map <- c("4X", "8X", "16X", "32X", "64X", "128X")
control_metadata$Test_Dataset <- value_map[control_metadata$Test_Dataset]

#### Create heatmap

In [12]:
# Annotation objects

# Define the desired order for Peak Coverage levels
peak_coverage_order <- c("4X", "8X", "16X", "32X", "64X", "128X")

# Define the desired order for Endedness and Peak Type
desired_column_order <- c(
  "paired_broad_bowtie2", "paired_broad_bwa_mem", "paired_broad_STAR",
  "paired_narrow_bowtie2", "paired_narrow_bwa_mem", "paired_narrow_STAR",
  "single_broad_bowtie2", "single_broad_bwa_mem", "single_broad_STAR",
  "single_narrow_bowtie2", "single_narrow_bwa_mem", "single_narrow_STAR"
)

# Rearrange the columns in matrix
control_matrix_reordered <- control_matrix[, desired_column_order]

# Define custom grouping factor for column_split
n_blocks <- 4
n_columns_per_block <- 3
block_labels <- rep(1:n_blocks, each = n_columns_per_block)
block_split_factor <- factor(block_labels)

# Split rows by peak caller
row_split_factor <- factor(control_metadata$Peak_Caller)

# Row annotations
row_ha = rowAnnotation(`Peak Caller` = control_metadata$Peak_Caller,
                       `Deduplicator` = control_metadata$Deduplicator,
                       `Peak Coverage` = factor(control_metadata$Test_Dataset, levels = peak_coverage_order),
                       col = list(`Peak Caller` = c("Cisgenome" = "#c2ffc3","Genrich"= "#90ec7c","MACS3"= "#00b22a", "PePr" = "#006b05"),
                                  `Deduplicator` = c("Samtools" = "#620093", "None" = "#e6e6fa", "Picard" = "#aea3e0", "Sambamba" = "#835fbd"),
                                  `Peak Coverage` = c("4X" = "#00CCFF", "8X" = "#00A3FF", "16X" = "#007AFF", "32X" = "#0052FF", "64X" = "#0000FF", "128X" = "#0700C4"))
                        )

# Column annotations
col_ha = HeatmapAnnotation(`Endedness` = c(rep("Paired", 6), rep("Single", 6)),
                            `Peak Type` = c(rep("Broad", 3), rep("Narrow", 3), rep("Broad", 3), rep("Narrow", 3)),
                            `Aligner` = c("Bowtie2", "BWA-MEM", "STAR", "Bowtie2", "BWA-MEM", "STAR", "Bowtie2", "BWA-MEM", "STAR", "Bowtie2", "BWA-MEM", "STAR"),
                            col = list(`Endedness` = c("Paired" = "#f7d3db", "Single" = "#eb647e"),
                                       `Peak Type` = c("Narrow" = "#ff4d00", "Broad" = "#ff9a00"),
                                       `Aligner` = c("BWA-MEM" = "#8c0000", "Bowtie2" = "#ff0000", "STAR" = "#400000"))
                            )

In [13]:
# Create heatmap

pdf("tables_and_figures/precision_control_heatmap.pdf", width = 6, height = 10)

# Heatmap for control
control_heatmap_object <- Heatmap(
  # Data
  control_matrix, 
  
  # Column customizations
  cluster_columns = FALSE,
  show_column_names = FALSE,
  column_title = NULL,
  top_annotation = col_ha,
  column_split = block_split_factor,
  column_gap = unit(1, "mm"),
    
  # Row customizations
  cluster_rows = FALSE,
  show_row_names = FALSE,
  row_title = NULL,
  left_annotation = row_ha,
  row_split = row_split_factor,
  row_gap = unit(1, "mm"),

  # Legend customizations
  heatmap_legend_param = list(title = "Precision"),
  col = colorRampPalette(c("#B2182B", "#EEEEEE", "#2166AC"))(100),

  # Other customizations
  border = FALSE, 
  width = ncol(control_matrix)*unit(5, "mm"), 
  height = nrow(control_matrix)*unit(2, "mm")
)

ht = draw(control_heatmap_object)

# Print width and height to determine appropriate export size
w = ComplexHeatmap:::width(ht)
w = convertX(w, "inch", valueOnly = TRUE)
h = ComplexHeatmap:::height(ht)
h = convertY(h, "inch", valueOnly = TRUE)
c(w, h)

dev.off()

[1] 5.347920 9.540227

png 
  2

### Sensitivity

#### Create matrix

In [14]:
# Subset for data with control
control_df <- heatmap_data[heatmap_data$Control == "with_control", ]

# Melt the data from the heatmap_data dataframe
melted_control_df <- melt(control_df, id.vars = c("x_axis_labels", "y_axis_labels"), measure.vars = "Sensitivity")

# Cast the data into the desired format
cast_control_df <- dcast(melted_control_df, y_axis_labels ~ x_axis_labels, value.var = "value")

# Generate matrices
rownames(cast_control_df) <- cast_control_df$y_axis_labels
cast_control_df$y_axis_labels <- NULL
control_matrix <- as.matrix(cast_control_df)

# Transpose matrix to switch rows and columns
control_matrix <- t(control_matrix)

#### Prepare metadata 

In [15]:
# Save matrix as data frame for manipulation
control_metadata <- as.data.frame(control_matrix)

# Peak Caller
new_row_names <- sub("^(.*?)_.*$", "\\1", rownames(control_matrix))
replacement_rules <- c("cisgenome" = "Cisgenome", "macs3" = "MACS3", "pepr" = "PePr", "genrich" = "Genrich")
new_row_names <- sapply(new_row_names, function(name) {
  if (name %in% names(replacement_rules)) {
    return(replacement_rules[name])
  } else {
    return(name)
  }
})
control_metadata$Peak_Caller <- new_row_names

# Deduplicator
new_row_names <- str_extract(rownames(control_matrix), "(?<=_)[^_]+")
replacement_rules <- c("no" = "None", "sambamba" = "Sambamba", "picard" = "Picard", "samtools" = "Samtools")
new_row_names <- sapply(new_row_names, function(name) {
  if (name %in% names(replacement_rules)) {
    return(replacement_rules[name])
  } else {
    return(name)
  }
})
control_metadata$Deduplicator <- new_row_names

# Test_Dataset
control_metadata$Test_Dataset <- as.integer(gsub(".*_(\\d+)$", "\\1", rownames(control_metadata)))
value_map <- c("4X", "8X", "16X", "32X", "64X", "128X")
control_metadata$Test_Dataset <- value_map[control_metadata$Test_Dataset]

#### Create heatmap

In [16]:
# Annotation objects

# Define the desired order for Peak Coverage levels
peak_coverage_order <- c("4X", "8X", "16X", "32X", "64X", "128X")

# Define the desired order for Endedness and Peak Type
desired_column_order <- c(
  "paired_broad_bowtie2", "paired_broad_bwa_mem", "paired_broad_STAR",
  "paired_narrow_bowtie2", "paired_narrow_bwa_mem", "paired_narrow_STAR",
  "single_broad_bowtie2", "single_broad_bwa_mem", "single_broad_STAR",
  "single_narrow_bowtie2", "single_narrow_bwa_mem", "single_narrow_STAR"
)

# Rearrange the columns in matrix
control_matrix_reordered <- control_matrix[, desired_column_order]

# Define custom grouping factor for column_split
n_blocks <- 4
n_columns_per_block <- 3
block_labels <- rep(1:n_blocks, each = n_columns_per_block)
block_split_factor <- factor(block_labels)

# Split rows by peak caller
row_split_factor <- factor(control_metadata$Peak_Caller)

# Row annotations
row_ha = rowAnnotation(`Peak Caller` = control_metadata$Peak_Caller,
                       `Deduplicator` = control_metadata$Deduplicator,
                       `Peak Coverage` = factor(control_metadata$Test_Dataset, levels = peak_coverage_order),
                       col = list(`Peak Caller` = c("Cisgenome" = "#c2ffc3","Genrich"= "#90ec7c","MACS3"= "#00b22a", "PePr" = "#006b05"),
                                  `Deduplicator` = c("Samtools" = "#620093", "None" = "#e6e6fa", "Picard" = "#aea3e0", "Sambamba" = "#835fbd"),
                                  `Peak Coverage` = c("4X" = "#00CCFF", "8X" = "#00A3FF", "16X" = "#007AFF", "32X" = "#0052FF", "64X" = "#0000FF", "128X" = "#0700C4"))
                        )

# Column annotations
col_ha = HeatmapAnnotation(`Endedness` = c(rep("Paired", 6), rep("Single", 6)),
                            `Peak Type` = c(rep("Broad", 3), rep("Narrow", 3), rep("Broad", 3), rep("Narrow", 3)),
                            `Aligner` = c("Bowtie2", "BWA-MEM", "STAR", "Bowtie2", "BWA-MEM", "STAR", "Bowtie2", "BWA-MEM", "STAR", "Bowtie2", "BWA-MEM", "STAR"),
                            col = list(`Endedness` = c("Paired" = "#f7d3db", "Single" = "#eb647e"),
                                       `Peak Type` = c("Narrow" = "#ff4d00", "Broad" = "#ff9a00"),
                                       `Aligner` = c("BWA-MEM" = "#8c0000", "Bowtie2" = "#ff0000", "STAR" = "#400000"))
                            )

In [17]:
# Create heatmap

pdf("tables_and_figures/sensitivity_control_heatmap.pdf", width = 6, height = 10)

# Heatmap for control
control_heatmap_object <- Heatmap(
  # Data
  control_matrix, 
  
  # Column customizations
  cluster_columns = FALSE,
  show_column_names = FALSE,
  column_title = NULL,
  top_annotation = col_ha,
  column_split = block_split_factor,
  column_gap = unit(1, "mm"),
    
  # Row customizations
  cluster_rows = FALSE,
  show_row_names = FALSE,
  row_title = NULL,
  left_annotation = row_ha,
  row_split = row_split_factor,
  row_gap = unit(1, "mm"),

  # Legend customizations
  heatmap_legend_param = list(title = "Sensitivity"),
  col = colorRampPalette(c("#B2182B", "#EEEEEE", "#2166AC"))(100),

  # Other customizations
  border = FALSE, 
  width = ncol(control_matrix)*unit(5, "mm"), 
  height = nrow(control_matrix)*unit(2, "mm")
)

ht = draw(control_heatmap_object)

# Print width and height to determine appropriate export size
w = ComplexHeatmap:::width(ht)
w = convertX(w, "inch", valueOnly = TRUE)
h = ComplexHeatmap:::height(ht)
h = convertY(h, "inch", valueOnly = TRUE)
c(w, h)

dev.off()

[1] 5.347920 9.540227

png 
  2

## Data with no Control

### F1 Scores

#### Create matrix

In [18]:
# Subset for data with no control
no_control_df <- heatmap_data[heatmap_data$Control == "no_control", ]

# Melt the data from the heatmap_data dataframe
melted_no_control_df <- melt(no_control_df, id.vars = c("x_axis_labels", "y_axis_labels"), measure.vars = "F1_Score")

# Cast the data into the desired format
cast_no_control_df <- dcast(melted_no_control_df, y_axis_labels ~ x_axis_labels, value.var = "value")

# Generate matrices
rownames(cast_no_control_df) <- cast_no_control_df$y_axis_labels
cast_no_control_df$y_axis_labels <- NULL
no_control_matrix <- as.matrix(cast_no_control_df)

# Transpose matrix to switch rows and columns
no_control_matrix <- t(no_control_matrix)

#### Prepare metadata 

In [19]:
# Save matrix as data frame for manipulation
no_control_metadata <- as.data.frame(no_control_matrix)

# Peak Caller
new_row_names <- sub("^(.*?)_.*$", "\\1", rownames(no_control_matrix))
replacement_rules <- c("cisgenome" = "Cisgenome", "macs3" = "MACS3", "pepr" = "PePr", "genrich" = "Genrich")
new_row_names <- sapply(new_row_names, function(name) {
  if (name %in% names(replacement_rules)) {
    return(replacement_rules[name])
  } else {
    return(name)
  }
})
no_control_metadata$Peak_Caller <- new_row_names

# Deduplicator
new_row_names <- str_extract(rownames(no_control_matrix), "(?<=_)[^_]+")
replacement_rules <- c("no" = "None", "sambamba" = "Sambamba", "picard" = "Picard", "samtools" = "Samtools")
new_row_names <- sapply(new_row_names, function(name) {
  if (name %in% names(replacement_rules)) {
    return(replacement_rules[name])
  } else {
    return(name)
  }
})
no_control_metadata$Deduplicator <- new_row_names

# Test_Dataset
no_control_metadata$Test_Dataset <- as.integer(gsub(".*_(\\d+)$", "\\1", rownames(no_control_metadata)))
value_map <- c("4X", "8X", "16X", "32X", "64X", "128X")
no_control_metadata$Test_Dataset <- value_map[no_control_metadata$Test_Dataset]

#### Create heatmap

In [20]:
# Annotation objects

# Define the desired order for Peak Coverage levels
peak_coverage_order <- c("4X", "8X", "16X", "32X", "64X", "128X")

# Define the desired order for Endedness and Peak Type
desired_column_order <- c(
  "paired_broad_bowtie2", "paired_broad_bwa_mem", "paired_broad_STAR",
  "paired_narrow_bowtie2", "paired_narrow_bwa_mem", "paired_narrow_STAR",
  "single_broad_bowtie2", "single_broad_bwa_mem", "single_broad_STAR",
  "single_narrow_bowtie2", "single_narrow_bwa_mem", "single_narrow_STAR"
)

# Rearrange the columns in matrix
no_control_matrix_reordered <- no_control_matrix[, desired_column_order]

# Define custom grouping factor for column_split
n_blocks <- 4
n_columns_per_block <- 3
block_labels <- rep(1:n_blocks, each = n_columns_per_block)
block_split_factor <- factor(block_labels)

# Split rows by peak caller
row_split_factor <- factor(no_control_metadata$Peak_Caller)

# Row annotations
row_ha = rowAnnotation(`Peak Caller` = no_control_metadata$Peak_Caller,
                       `Deduplicator` = no_control_metadata$Deduplicator,
                       `Peak Coverage` = factor(no_control_metadata$Test_Dataset, levels = peak_coverage_order),
                       col = list(`Peak Caller` = c("Cisgenome" = "#c2ffc3","Genrich"= "#90ec7c","MACS3"= "#00b22a", "PePr" = "#006b05"),
                                  `Deduplicator` = c("Samtools" = "#620093", "None" = "#e6e6fa", "Picard" = "#aea3e0", "Sambamba" = "#835fbd"),
                                  `Peak Coverage` = c("4X" = "#00CCFF", "8X" = "#00A3FF", "16X" = "#007AFF", "32X" = "#0052FF", "64X" = "#0000FF", "128X" = "#0700C4"))
                        )

# Column annotations
col_ha = HeatmapAnnotation(`Endedness` = c(rep("Paired", 6), rep("Single", 6)),
                            `Peak Type` = c(rep("Broad", 3), rep("Narrow", 3), rep("Broad", 3), rep("Narrow", 3)),
                            `Aligner` = c("Bowtie2", "BWA-MEM", "STAR", "Bowtie2", "BWA-MEM", "STAR", "Bowtie2", "BWA-MEM", "STAR", "Bowtie2", "BWA-MEM", "STAR"),
                            col = list(`Endedness` = c("Paired" = "#f7d3db", "Single" = "#eb647e"),
                                       `Peak Type` = c("Narrow" = "#ff4d00", "Broad" = "#ff9a00"),
                                       `Aligner` = c("BWA-MEM" = "#8c0000", "Bowtie2" = "#ff0000", "STAR" = "#400000"))
                            )

In [21]:
# Create heatmap

pdf("tables_and_figures/f1_score_no_control_heatmap.pdf", width = 6, height = 10)

# Heatmap for control
no_control_heatmap_object <- Heatmap(
  # Data
  no_control_matrix, 
  
  # Column customizations
  cluster_columns = FALSE,
  show_column_names = FALSE,
  column_title = NULL,
  top_annotation = col_ha,
  column_split = block_split_factor,
  column_gap = unit(1, "mm"),
    
  # Row customizations
  cluster_rows = FALSE,
  show_row_names = FALSE,
  row_title = NULL,
  left_annotation = row_ha,
  row_split = row_split_factor,
  row_gap = unit(1, "mm"),

  # Legend customizations
  heatmap_legend_param = list(title = "F1 Score"),
  col = colorRampPalette(c("#B2182B", "#EEEEEE", "#2166AC"))(100),

  # Other customizations
  border = FALSE, 
  width = ncol(no_control_matrix)*unit(5, "mm"), 
  height = nrow(no_control_matrix)*unit(2, "mm")
)

ht = draw(no_control_heatmap_object)

# Print width and height to determine appropriate export size
w = ComplexHeatmap:::width(ht)
w = convertX(w, "inch", valueOnly = TRUE)
h = ComplexHeatmap:::height(ht)
h = convertY(h, "inch", valueOnly = TRUE)
c(w, h)

dev.off()

[1] 5.347920 9.540227

png 
  2

### Precision

#### Create matrix

In [22]:
# Subset for data with no control
no_control_df <- heatmap_data[heatmap_data$Control == "no_control", ]

# Melt the data from the heatmap_data dataframe
melted_no_control_df <- melt(no_control_df, id.vars = c("x_axis_labels", "y_axis_labels"), measure.vars = "Precision")

# Cast the data into the desired format
cast_no_control_df <- dcast(melted_no_control_df, y_axis_labels ~ x_axis_labels, value.var = "value")

# Generate matrices
rownames(cast_no_control_df) <- cast_no_control_df$y_axis_labels
cast_no_control_df$y_axis_labels <- NULL
no_control_matrix <- as.matrix(cast_no_control_df)

# Transpose matrix to switch rows and columns
no_control_matrix <- t(no_control_matrix)

#### Prepare metadata

In [23]:
# Save matrix as data frame for manipulation
no_control_metadata <- as.data.frame(no_control_matrix)

# Peak Caller
new_row_names <- sub("^(.*?)_.*$", "\\1", rownames(no_control_matrix))
replacement_rules <- c("cisgenome" = "Cisgenome", "macs3" = "MACS3", "pepr" = "PePr", "genrich" = "Genrich")
new_row_names <- sapply(new_row_names, function(name) {
  if (name %in% names(replacement_rules)) {
    return(replacement_rules[name])
  } else {
    return(name)
  }
})
no_control_metadata$Peak_Caller <- new_row_names

# Deduplicator
new_row_names <- str_extract(rownames(no_control_matrix), "(?<=_)[^_]+")
replacement_rules <- c("no" = "None", "sambamba" = "Sambamba", "picard" = "Picard", "samtools" = "Samtools")
new_row_names <- sapply(new_row_names, function(name) {
  if (name %in% names(replacement_rules)) {
    return(replacement_rules[name])
  } else {
    return(name)
  }
})
no_control_metadata$Deduplicator <- new_row_names

# Test_Dataset
no_control_metadata$Test_Dataset <- as.integer(gsub(".*_(\\d+)$", "\\1", rownames(no_control_metadata)))
value_map <- c("4X", "8X", "16X", "32X", "64X", "128X")
no_control_metadata$Test_Dataset <- value_map[no_control_metadata$Test_Dataset]

#### Create heatmap

In [24]:
# Annotation objects

# Define the desired order for Peak Coverage levels
peak_coverage_order <- c("4X", "8X", "16X", "32X", "64X", "128X")

# Define the desired order for Endedness and Peak Type
desired_column_order <- c(
  "paired_broad_bowtie2", "paired_broad_bwa_mem", "paired_broad_STAR",
  "paired_narrow_bowtie2", "paired_narrow_bwa_mem", "paired_narrow_STAR",
  "single_broad_bowtie2", "single_broad_bwa_mem", "single_broad_STAR",
  "single_narrow_bowtie2", "single_narrow_bwa_mem", "single_narrow_STAR"
)

# Rearrange the columns in matrix
no_control_matrix_reordered <- no_control_matrix[, desired_column_order]

# Define custom grouping factor for column_split
n_blocks <- 4
n_columns_per_block <- 3
block_labels <- rep(1:n_blocks, each = n_columns_per_block)
block_split_factor <- factor(block_labels)

# Split rows by peak caller
row_split_factor <- factor(no_control_metadata$Peak_Caller)

# Row annotations
row_ha = rowAnnotation(`Peak Caller` = no_control_metadata$Peak_Caller,
                       `Deduplicator` = no_control_metadata$Deduplicator,
                       `Peak Coverage` = factor(no_control_metadata$Test_Dataset, levels = peak_coverage_order),
                       col = list(`Peak Caller` = c("Cisgenome" = "#c2ffc3","Genrich"= "#90ec7c","MACS3"= "#00b22a", "PePr" = "#006b05"),
                                  `Deduplicator` = c("Samtools" = "#620093", "None" = "#e6e6fa", "Picard" = "#aea3e0", "Sambamba" = "#835fbd"),
                                  `Peak Coverage` = c("4X" = "#00CCFF", "8X" = "#00A3FF", "16X" = "#007AFF", "32X" = "#0052FF", "64X" = "#0000FF", "128X" = "#0700C4"))
                        )

# Column annotations
col_ha = HeatmapAnnotation(`Endedness` = c(rep("Paired", 6), rep("Single", 6)),
                            `Peak Type` = c(rep("Broad", 3), rep("Narrow", 3), rep("Broad", 3), rep("Narrow", 3)),
                            `Aligner` = c("Bowtie2", "BWA-MEM", "STAR", "Bowtie2", "BWA-MEM", "STAR", "Bowtie2", "BWA-MEM", "STAR", "Bowtie2", "BWA-MEM", "STAR"),
                            col = list(`Endedness` = c("Paired" = "#f7d3db", "Single" = "#eb647e"),
                                       `Peak Type` = c("Narrow" = "#ff4d00", "Broad" = "#ff9a00"),
                                       `Aligner` = c("BWA-MEM" = "#8c0000", "Bowtie2" = "#ff0000", "STAR" = "#400000"))
                            )

In [25]:
# Create heatmap

pdf("tables_and_figures/precision_no_control_heatmap.pdf", width = 6, height = 10)

# Heatmap for control
no_control_heatmap_object <- Heatmap(
  # Data
  no_control_matrix, 
  
  # Column customizations
  cluster_columns = FALSE,
  show_column_names = FALSE,
  column_title = NULL,
  top_annotation = col_ha,
  column_split = block_split_factor,
  column_gap = unit(1, "mm"),
    
  # Row customizations
  cluster_rows = FALSE,
  show_row_names = FALSE,
  row_title = NULL,
  left_annotation = row_ha,
  row_split = row_split_factor,
  row_gap = unit(1, "mm"),

  # Legend customizations
  heatmap_legend_param = list(title = "Precision"),
  col = colorRampPalette(c("#B2182B", "#EEEEEE", "#2166AC"))(100),

  # Other customizations
  border = FALSE, 
  width = ncol(no_control_matrix)*unit(5, "mm"), 
  height = nrow(no_control_matrix)*unit(2, "mm")
)

ht = draw(no_control_heatmap_object)

# Print width and height to determine appropriate export size
w = ComplexHeatmap:::width(ht)
w = convertX(w, "inch", valueOnly = TRUE)
h = ComplexHeatmap:::height(ht)
h = convertY(h, "inch", valueOnly = TRUE)
c(w, h)

dev.off()

[1] 5.347920 9.540227

png 
  2

### Sensitivity

#### Create matrix

In [26]:
# Subset for data with no control
no_control_df <- heatmap_data[heatmap_data$Control == "no_control", ]

# Melt the data from the heatmap_data dataframe
melted_no_control_df <- melt(no_control_df, id.vars = c("x_axis_labels", "y_axis_labels"), measure.vars = "Sensitivity")

# Cast the data into the desired format
cast_no_control_df <- dcast(melted_no_control_df, y_axis_labels ~ x_axis_labels, value.var = "value")

# Generate matrices
rownames(cast_no_control_df) <- cast_no_control_df$y_axis_labels
cast_no_control_df$y_axis_labels <- NULL
no_control_matrix <- as.matrix(cast_no_control_df)

# Transpose matrix to switch rows and columns
no_control_matrix <- t(no_control_matrix)

#### Prepare metadata

In [27]:
# Save matrix as data frame for manipulation
no_control_metadata <- as.data.frame(no_control_matrix)

# Peak Caller
new_row_names <- sub("^(.*?)_.*$", "\\1", rownames(no_control_matrix))
replacement_rules <- c("cisgenome" = "Cisgenome", "macs3" = "MACS3", "pepr" = "PePr", "genrich" = "Genrich")
new_row_names <- sapply(new_row_names, function(name) {
  if (name %in% names(replacement_rules)) {
    return(replacement_rules[name])
  } else {
    return(name)
  }
})
no_control_metadata$Peak_Caller <- new_row_names

# Deduplicator
new_row_names <- str_extract(rownames(no_control_matrix), "(?<=_)[^_]+")
replacement_rules <- c("no" = "None", "sambamba" = "Sambamba", "picard" = "Picard", "samtools" = "Samtools")
new_row_names <- sapply(new_row_names, function(name) {
  if (name %in% names(replacement_rules)) {
    return(replacement_rules[name])
  } else {
    return(name)
  }
})
no_control_metadata$Deduplicator <- new_row_names

# Test_Dataset
no_control_metadata$Test_Dataset <- as.integer(gsub(".*_(\\d+)$", "\\1", rownames(no_control_metadata)))
value_map <- c("4X", "8X", "16X", "32X", "64X", "128X")
no_control_metadata$Test_Dataset <- value_map[no_control_metadata$Test_Dataset]

#### Create heatmap

In [28]:
# Annotation objects

# Define the desired order for Peak Coverage levels
peak_coverage_order <- c("4X", "8X", "16X", "32X", "64X", "128X")

# Define the desired order for Endedness and Peak Type
desired_column_order <- c(
  "paired_broad_bowtie2", "paired_broad_bwa_mem", "paired_broad_STAR",
  "paired_narrow_bowtie2", "paired_narrow_bwa_mem", "paired_narrow_STAR",
  "single_broad_bowtie2", "single_broad_bwa_mem", "single_broad_STAR",
  "single_narrow_bowtie2", "single_narrow_bwa_mem", "single_narrow_STAR"
)

# Rearrange the columns in matrix
no_control_matrix_reordered <- no_control_matrix[, desired_column_order]

# Define custom grouping factor for column_split
n_blocks <- 4
n_columns_per_block <- 3
block_labels <- rep(1:n_blocks, each = n_columns_per_block)
block_split_factor <- factor(block_labels)

# Split rows by peak caller
row_split_factor <- factor(no_control_metadata$Peak_Caller)

# Row annotations
row_ha = rowAnnotation(`Peak Caller` = no_control_metadata$Peak_Caller,
                       `Deduplicator` = no_control_metadata$Deduplicator,
                       `Peak Coverage` = factor(no_control_metadata$Test_Dataset, levels = peak_coverage_order),
                       col = list(`Peak Caller` = c("Cisgenome" = "#c2ffc3","Genrich"= "#90ec7c","MACS3"= "#00b22a", "PePr" = "#006b05"),
                                  `Deduplicator` = c("Samtools" = "#620093", "None" = "#e6e6fa", "Picard" = "#aea3e0", "Sambamba" = "#835fbd"),
                                  `Peak Coverage` = c("4X" = "#00CCFF", "8X" = "#00A3FF", "16X" = "#007AFF", "32X" = "#0052FF", "64X" = "#0000FF", "128X" = "#0700C4"))
                        )

# Column annotations
col_ha = HeatmapAnnotation(`Endedness` = c(rep("Paired", 6), rep("Single", 6)),
                            `Peak Type` = c(rep("Broad", 3), rep("Narrow", 3), rep("Broad", 3), rep("Narrow", 3)),
                            `Aligner` = c("Bowtie2", "BWA-MEM", "STAR", "Bowtie2", "BWA-MEM", "STAR", "Bowtie2", "BWA-MEM", "STAR", "Bowtie2", "BWA-MEM", "STAR"),
                            col = list(`Endedness` = c("Paired" = "#f7d3db", "Single" = "#eb647e"),
                                       `Peak Type` = c("Narrow" = "#ff4d00", "Broad" = "#ff9a00"),
                                       `Aligner` = c("BWA-MEM" = "#8c0000", "Bowtie2" = "#ff0000", "STAR" = "#400000"))
                            )

In [29]:
# Create heatmap

pdf("tables_and_figures/sensitivity_no_control_heatmap.pdf", width = 6, height = 10)

# Heatmap for control
no_control_heatmap_object <- Heatmap(
  # Data
  no_control_matrix, 
  
  # Column customizations
  cluster_columns = FALSE,
  show_column_names = FALSE,
  column_title = NULL,
  top_annotation = col_ha,
  column_split = block_split_factor,
  column_gap = unit(1, "mm"),
    
  # Row customizations
  cluster_rows = FALSE,
  show_row_names = FALSE,
  row_title = NULL,
  left_annotation = row_ha,
  row_split = row_split_factor,
  row_gap = unit(1, "mm"),

  # Legend customizations
  heatmap_legend_param = list(title = "Sensitivity"),
  col = colorRampPalette(c("#B2182B", "#EEEEEE", "#2166AC"))(100),

  # Other customizations
  border = FALSE, 
  width = ncol(no_control_matrix)*unit(5, "mm"), 
  height = nrow(no_control_matrix)*unit(2, "mm")
)

ht = draw(no_control_heatmap_object)

# Print width and height to determine appropriate export size
w = ComplexHeatmap:::width(ht)
w = convertX(w, "inch", valueOnly = TRUE)
h = ComplexHeatmap:::height(ht)
h = convertY(h, "inch", valueOnly = TRUE)
c(w, h)

dev.off()

[1] 5.347920 9.540227

png 
  2